In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import combinations
from scipy.stats import shapiro

In [2]:
venta = pd.read_csv("inmuebles_venta.csv")

In [3]:
df = venta.copy()


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9683 entries, 0 to 9682
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     9683 non-null   object
 1   Descripción            9683 non-null   object
 2   Localización           9683 non-null   object
 3   Enlace                 9683 non-null   object
 4   Precio                 9350 non-null   object
 5   Superficie Construida  9350 non-null   object
 6   Última Actualización   9350 non-null   object
 7   Consumo Energético     2078 non-null   object
 8   Emisiones CO2          2078 non-null   object
 9   Características        9350 non-null   object
 10  Tipo de operación      9683 non-null   object
 11  timestamp_scrapeo      9683 non-null   object
dtypes: object(12)
memory usage: 907.9+ KB


In [5]:
# Función para extraer las características de la columna y convertirlas en columnas separadas
def extraer_caracteristicas(fila):
    # Elimina los caracteres adicionales para convertir la cadena en una lista de tuplas
    fila = fila.strip("[]")  # Elimina los corchetes externos
    elementos = fila.split("), (")  # Divide la cadena en tuplas individuales
    
    datos_extraidos = {}
    for elemento in elementos:
        # Divide cada elemento en clave y valor
        clave, valor = elemento.split(", ", 1)
        clave = clave.replace("(", "").replace("'", "").strip()  # Limpia clave
        valor = valor.replace(")", "").replace("'", "").strip()  # Limpia valor
        datos_extraidos[clave.strip(':')] = valor  # Remueve ':' del final de la clave si está presente
    return datos_extraidos

# Aplicar la función a la columna 'Características' y expandir en columnas separadas
caracteristicas_df = df['Características'].dropna().apply(extraer_caracteristicas)
caracteristicas_expandidas = pd.json_normalize(caracteristicas_df)

# Combinar las columnas expandidas con el DataFrame original (elimina la columna 'Características' original)
df_limpio = pd.concat([df.drop(columns=['Características']), caracteristicas_expandidas], axis=1)

# Guardar el DataFrame limpio en un nuevo archivo CSV si deseas conservarlo
# df_limpio.to_csv('venta_inmuebles_limpio.csv', index=False)

# Visualización del resultado
print(df_limpio.head())


                                     id  \
0  bfcba5e4-5958-4445-9de7-0ab4df732b8c   
1  a0f3e8d5-ca6f-45c5-a5d4-2d3d6b7bbd22   
2  2efd9d2c-cdc9-45a6-91eb-6173f0e94ac7   
3  15c5ec6e-b28e-42d1-8fb9-497fa3158469   
4  a0676ed8-fa97-44ac-b242-aa947e8a2016   

                                         Descripción  \
0  Chalet en venta en Boadilla del Monte - Parque...   
1                    Casa pareada en venta en Cobeña   
2    Piso en venta en Calle Joaquim Blume, Número 14   
3  Piso en venta en Avenida de Concha Espina, cer...   
4                 Piso en venta en Calle del Duratón   

                                        Localización  \
0            Parque de Boadilla (Boadilla del Monte)   
1                                             Cobeña   
2                  Zona Suroeste (Torrejón de Ardoz)   
3  Hispanoamérica (Distrito Chamartín. Madrid Cap...   
4           Lucero (Distrito Latina. Madrid Capital)   

                                              Enlace     Precio  \


In [6]:
df_limpio

,id,Descripción,Localización,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Tipo de operación,...,Baños,Antigüedad,Conservación,Referencia,Tipo de casa,Superficie útil,Planta,Gastos de comunidad,Interior,Exterior
0,bfcba5e4-5958-4445-9de7-0ab4df732b8c,Chalet en venta en Boadilla del Monte - Parque...,Parque de Boadilla (Boadilla del Monte),https://www.pisos.com/comprar/chalet-parque_de...,1.795.000,415,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,compra,...,5,Entre 10 y 20 años,En buen estado,IF5271-181,NaN,NaN,NaN,NaN,NaN,NaN
1,a0f3e8d5-ca6f-45c5-a5d4-2d3d6b7bbd22,Casa pareada en venta en Cobeña,Cobeña,https://www.pisos.com/comprar/casa_pareada-cob...,670.000,360,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,compra,...,3,Entre 10 y 20 años,En buen estado,IF5271-206,Pareada,NaN,NaN,NaN,NaN,NaN
2,2efd9d2c-cdc9-45a6-91eb-6173f0e94ac7,"Piso en venta en Calle Joaquim Blume, Número 14",Zona Suroeste (Torrejón de Ardoz),https://www.pisos.com/comprar/piso-zona_suroes...,254.900,103,Anuncio actualizado el 02/11/2024,NaN,NaN,compra,...,2,Entre 20 y 30 años,En buen estado,4525678-000058,NaN,92 m²,1ª,Entre 60 y 80 €,NaN,NaN
3,15c5ec6e-b28e-42d1-8fb9-497fa3158469,"Piso en venta en Avenida de Concha Espina, cer...",Hispanoamérica (Distrito Chamartín. Madrid Cap...,https://www.pisos.com/comprar/piso-chamartin_h...,950.000,141,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:56 Kg CO₂/m² año,compra,...,2,Más de 50 años,En buen estado,IF5271-51,NaN,NaN,1ª,NaN,NaN,NaN
4,a0676ed8-fa97-44ac-b242-aa947e8a2016,Piso en venta en Calle del Duratón,Lucero (Distrito Latina. Madrid Capital),https://www.pisos.com/comprar/piso-lucero28011...,210.000,65,Anuncio actualizado el 17/10/2024,NaN,NaN,compra,...,1,NaN,NaN,STA16-RP1212024128799,NaN,59 m²,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9678,ed977a06-35d6-4d05-bed2-b3a7afc07290,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9679,9796437d-4616-43fd-a0c5-590a338d2678,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9680,f5c76a54-c3bf-445d-ba23-4b516989214d,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9681,99ddc50b-86a9-4e62-8119-f8aa293ca866,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9683 entries, 0 to 9682
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     9683 non-null   object
 1   Descripción            9683 non-null   object
 2   Localización           9683 non-null   object
 3   Enlace                 9683 non-null   object
 4   Precio                 9350 non-null   object
 5   Superficie Construida  9350 non-null   object
 6   Última Actualización   9350 non-null   object
 7   Consumo Energético     2078 non-null   object
 8   Emisiones CO2          2078 non-null   object
 9   Características        9350 non-null   object
 10  Tipo de operación      9683 non-null   object
 11  timestamp_scrapeo      9683 non-null   object
dtypes: object(12)
memory usage: 907.9+ KB


In [8]:
# Lista de tipos de casa que queremos identificar
tipos_casa = ["Piso", "Chalet", "Adosado", "Casa", "Estudio", "Dúplex"]

# Función para extraer el tipo de casa de la descripción
def extraer_tipo_casa(descripcion):
    for tipo in tipos_casa:
        if tipo.lower() in descripcion.lower():
            return tipo
    return "Otro"  # Valor por defecto si no coincide con ninguno de los tipos en la lista

# Crear una nueva columna en el DataFrame con el tipo de casa
df_limpio['Tipo de Casa'] = df_limpio['Descripción'].apply(extraer_tipo_casa)

# Visualización del resultado
print(df_limpio[['Descripción', 'Tipo de Casa']].head())


                                         Descripción Tipo de Casa
0  Chalet en venta en Boadilla del Monte - Parque...       Chalet
1                    Casa pareada en venta en Cobeña         Casa
2    Piso en venta en Calle Joaquim Blume, Número 14         Piso
3  Piso en venta en Avenida de Concha Espina, cer...         Piso
4                 Piso en venta en Calle del Duratón         Piso


In [11]:
# Extraer la fecha de la columna 'Última Actualización' y convertirla a tipo datetime
df_limpio['Última Actualización'] = df_limpio['Última Actualización'].str.extract(r'(\d{2}/\d{2}/\d{4})')
df_limpio['Última Actualización'] = pd.to_datetime(df_limpio['Última Actualización'], format='%d/%m/%Y')

# Verificar el resultado
print(df_limpio['Última Actualización'].head())


AttributeError: Can only use .str accessor with string values!

In [12]:
df_limpio = df_limpio.drop(columns=["Tipo de casa","Referencia","Gastos de comunidad","Superficie solar","Interior","Exterior"])

In [13]:
# Eliminar filas donde 'Precio' es NaN
df_limpio = df_limpio.dropna(subset=['Precio'])


In [14]:
# Filtrar filas en 'Precio' que contienen caracteres no numéricos
valores_no_numericos = df_limpio[~df_limpio['Precio'].str.replace('.', '', regex=False).str.replace(',', '', regex=False).str.isnumeric()]

# Mostrar los resultados
print(valores_no_numericos[['Precio']])


     Precio
601       A
930       A
1183      A
1199      A
1321      A
...     ...
9324      A
9325      A
9329      A
9330      A
9331      A

[61 rows x 1 columns]


In [15]:
df_limpio['Precio'] = pd.to_numeric(
    df_limpio['Precio']
    .astype(str)                       # Convertir a string para aplicar reemplazos
    .str.replace(r'[^\d]', '', regex=True),  # Eliminar todos los caracteres no numéricos
    errors='coerce'
)

# Eliminar filas donde 'Precio' es NaN (valores no convertibles)
df_limpio = df_limpio.dropna(subset=['Precio'])

# Convertir 'Precio' a entero después de limpiar
df_limpio['Precio'] = df_limpio['Precio'].astype(float)

# Verificar el resultado
print(df_limpio['Precio'].head())

0    1795000.0
1     670000.0
2     254900.0
3     950000.0
4     210000.0
Name: Precio, dtype: float64


In [16]:
# Rellenar valores nulos en columnas categóricas
df_limpio['Antigüedad'].fillna('No especificado', inplace=True)
df_limpio['Conservación'].fillna('No especificado', inplace=True)
df_limpio['Planta'].fillna('No especificado', inplace=True)

/var/folders/jb/v4t6swb9003c33x4phtghtpc0000gn/T/ipykernel_59842/911444348.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpio['Antigüedad'].fillna('No especificado', inplace=True)
/var/folders/jb/v4t6swb9003c33x4phtghtpc0000gn/T/ipykernel_59842/911444348.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

In [17]:
df_limpio['Superficie Construida'] = (
    df_limpio['Superficie Construida']
    .astype(str)                        # Convertir a string
    .str.replace(' m²', '', regex=False) # Eliminar ' m²' si existe
    .str.replace('.', '')               # Eliminar puntos de miles si existen
    .str.replace(',', '.', regex=False) # Cambiar coma decimal a punto (si aplica)
)

# Reemplazar los valores no numéricos con NaN y convertir el resto a float
df_limpio['Superficie Construida'] = pd.to_numeric(df_limpio['Superficie Construida'], errors='coerce')

# Convertir los valores a enteros, eliminando filas donde 'Superficie Construida' es NaN
df_limpio = df_limpio.dropna(subset=['Superficie Construida'])
df_limpio['Superficie Construida'] = df_limpio['Superficie Construida'].astype(float)


# Verificar el resultado
print(df_limpio[['Superficie Construida']].head())

   Superficie Construida
0                  415.0
1                  360.0
2                  103.0
3                  141.0
4                   65.0


In [18]:
df_limpio['Baños'] = pd.to_numeric(df_limpio['Baños'], errors='coerce').fillna(0).astype(int)
df_limpio['Habitaciones'] = pd.to_numeric(df_limpio['Habitaciones'], errors='coerce').fillna(0).astype(int)

In [19]:
# Eliminar filas duplicadas
df_limpio.drop_duplicates(inplace=True)

In [20]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9272 entries, 0 to 9349
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     9272 non-null   object        
 1   Descripción            9272 non-null   object        
 2   Localización           9272 non-null   object        
 3   Enlace                 9272 non-null   object        
 4   Precio                 9272 non-null   float64       
 5   Superficie Construida  9272 non-null   float64       
 6   Última Actualización   9272 non-null   datetime64[ns]
 7   Consumo Energético     2069 non-null   object        
 8   Emisiones CO2          2069 non-null   object        
 9   Tipo de operación      9272 non-null   object        
 10  timestamp_scrapeo      9272 non-null   object        
 11  Superficie construida  9244 non-null   object        
 12  Habitaciones           9272 non-null   int64         
 13  Baños   

In [21]:
# Usar .title() para capitalizar la primera letra de cada palabra en los nombres de columna
df_limpio.columns = df_limpio.columns.str.title()

# Verificar los nombres de las columnas
print(df_limpio.columns)


Index(['Id', 'Descripción', 'Localización', 'Enlace', 'Precio',
       'Superficie Construida', 'Última Actualización', 'Consumo Energético',
       'Emisiones Co2', 'Tipo De Operación', 'Timestamp_Scrapeo',
       'Superficie Construida', 'Habitaciones', 'Baños', 'Antigüedad',
       'Conservación', 'Superficie Útil', 'Planta', 'Tipo De Casa'],
      dtype='object')


In [22]:
df_limpio.to_csv('venta_inmuebles_limpio.csv', index=False)